In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score

In [2]:
train_df = pd.read_csv('train_embeddings_and_targets.csv')

X_train = train_df.drop(columns=['target']).values
y_train = train_df['target'].values

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

X_test = pd.read_csv('test_embeddings.csv').values

X_test_tensor = torch.FloatTensor(X_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim=768, hidden_dim1=256, hidden_dim2=64):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(hidden_dim2, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model = SimpleMLP(input_dim=X_train.shape[1])
device = torch.device('cuda')
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [4]:
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}')

Epoch 1/50, Loss: 0.4980
Epoch 2/50, Loss: 0.4433
Epoch 3/50, Loss: 0.4287
Epoch 4/50, Loss: 0.4207
Epoch 5/50, Loss: 0.4067
Epoch 6/50, Loss: 0.4020
Epoch 7/50, Loss: 0.3963
Epoch 8/50, Loss: 0.3952
Epoch 9/50, Loss: 0.3858
Epoch 10/50, Loss: 0.3802
Epoch 11/50, Loss: 0.3735
Epoch 12/50, Loss: 0.3653
Epoch 13/50, Loss: 0.3632
Epoch 14/50, Loss: 0.3562
Epoch 15/50, Loss: 0.3510
Epoch 16/50, Loss: 0.3437
Epoch 17/50, Loss: 0.3270
Epoch 18/50, Loss: 0.3244
Epoch 19/50, Loss: 0.3138
Epoch 20/50, Loss: 0.3068
Epoch 21/50, Loss: 0.2989
Epoch 22/50, Loss: 0.2943
Epoch 23/50, Loss: 0.2852
Epoch 24/50, Loss: 0.2809
Epoch 25/50, Loss: 0.2725
Epoch 26/50, Loss: 0.2668
Epoch 27/50, Loss: 0.2555
Epoch 28/50, Loss: 0.2506
Epoch 29/50, Loss: 0.2515
Epoch 30/50, Loss: 0.2447
Epoch 31/50, Loss: 0.2385
Epoch 32/50, Loss: 0.2256
Epoch 33/50, Loss: 0.2205
Epoch 34/50, Loss: 0.2171
Epoch 35/50, Loss: 0.2188
Epoch 36/50, Loss: 0.2096
Epoch 37/50, Loss: 0.1899
Epoch 38/50, Loss: 0.1886
Epoch 39/50, Loss: 0.

In [6]:
model.eval()
y_pred = []

with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs[0].to(device)
        outputs = model(inputs)
        predicted = (torch.sigmoid(outputs) >= 0.5).float()
        y_pred.extend(predicted.cpu().numpy().flatten())

y_pred = np.array(y_pred).astype(int)

sample_submission_df = pd.read_csv('sample_submission.csv')
submission_df = pd.DataFrame({
    'id': sample_submission_df['id'],
    'target': y_pred
})
submission_df.to_csv('submission_new.csv', index=False)